# 02 Validation Checker

**Purpose**: Test bundle validation interactively - see errors explained.

This is a PLAYGROUND, not a tutorial. For learning, see:
- `docs/VALIDATION.md` - How validation works
- `docs/TROUBLESHOOTING.md` - Common errors and fixes

---

In [ ]:
# Setup - run this cell first
import yaml
from amplifier_foundation import Bundle
from amplifier_foundation import validate_bundle

## Paste Your Bundle YAML

Edit the YAML below and run validation to see any errors or warnings.

In [ ]:
# Edit this YAML - try introducing errors to see validation messages
bundle_yaml = """
bundle:
  name: test-validation
  version: 1.0.0

session:
  orchestrator:
    module: loop-streaming
  context:
    module: context-simple

providers:
  - module: provider-anthropic
    source: git+https://github.com/microsoft/amplifier-module-provider-anthropic@main
    config:
      default_model: claude-sonnet-4-5

tools:
  - module: tool-filesystem
    source: git+https://github.com/microsoft/amplifier-module-tool-filesystem@main
"""

# Parse YAML
data = yaml.safe_load(bundle_yaml)
if not isinstance(data, dict):
    raise ValueError("Bundle YAML must parse to a dict")
bundle = Bundle.from_dict(data)
print(f"Parsed: {bundle.name} v{bundle.version}")

## Run Validation

In [ ]:
# Validate the bundle
result = validate_bundle(bundle)

print(f"Valid: {result.valid}")
print(f"Errors: {len(result.errors)}")
print(f"Warnings: {len(result.warnings)}")

In [ ]:
# Show errors (if any)
if result.errors:
    print("ERRORS (must fix):")
    print("=" * 50)
    for error in result.errors:
        print(f"❌ {error}")
else:
    print("✅ No errors!")

In [ ]:
# Show warnings (if any)
if result.warnings:
    print("WARNINGS (should review):")
    print("=" * 50)
    for warning in result.warnings:
        print(f"⚠️  {warning}")
else:
    print("✅ No warnings!")

## Common Validation Errors

Try introducing these errors in the YAML above to see the validation messages:

### Missing Bundle Name
```yaml
# Remove bundle.name to see:
# ❌ Bundle must have a name
bundle:
  version: 1.0.0
```

### Missing Module Field
```yaml
providers:
  - config:
      model: claude-sonnet-4-5
# ❌ providers[0]: Missing required 'module' field
```

### Invalid Config Type
```yaml
providers:
  - module: provider-anthropic
    config:
      - model: claude-sonnet-4-5
# ❌ providers[0]: 'config' must be a dict, got list
```

### Non-Dict Module Entry
```yaml
providers:
  - provider-anthropic
# ❌ providers[0]: Must be a dict, got str
```

## Try Intentionally Invalid Bundles

In [ ]:
# Example: Bundle with missing source URLs (common issue)
no_source_bundle = Bundle(
    name="no-sources",
    version="1.0.0",
    providers=[
        {"module": "provider-anthropic"},  # No source!
    ],
    tools=[
        {"module": "tool-filesystem"},  # No source!
    ],
)

result = validate_bundle(no_source_bundle)
print(f"Valid: {result.valid}")
print(f"Warnings: {len(result.warnings)}")
for w in result.warnings:
    print(f"  ⚠️ {w}")

In [ ]:
# Example: Bundle with missing module field
missing_module_bundle = Bundle(
    name="missing-module-field",
    version="1.0.0",
    providers=[
        {"config": {"model": "claude-sonnet-4-5"}},  # No 'module' field!
    ],
)

result = validate_bundle(missing_module_bundle)
print(f"Valid: {result.valid}")
print(f"Errors: {len(result.errors)}")
for e in result.errors:
    print(f"  ❌ {e}")

---

## Experiment!

Try modifying the bundle YAML above to:
- Remove required fields
- Use invalid values
- Omit source URLs
- Use malformed version strings

See how validation catches issues before they cause runtime errors!